In [1]:
import os
import numpy as np
import cv2
import pandas as pd

In [2]:
def readfile(path, label):
    # label 是一個 boolean variable，代表需不需要回傳 y 值
    image_dir = sorted(os.listdir(path))
    x = np.zeros((len(image_dir), 128, 128, 3), dtype=np.uint8)
    y = np.zeros((len(image_dir)), dtype=np.uint8)
    for i, file in enumerate(image_dir):
        img = cv2.imread(os.path.join(path, file))
        x[i, :, :] = cv2.resize(img,(128, 128))
        if label:
          y[i] = int(file.split("_")[0])
    if label:
      return x, y
    else:
      return x

In [3]:
a=os.listdir(r'F:\mldata\hw3\food-11\training')

In [4]:
from keras.utils import np_utils

Using TensorFlow backend.


In [5]:
from keras.models import Model,load_model
from keras.layers import Dense,Dropout,Flatten,Input,Conv2D,MaxPool2D
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping
from keras.models import load_model
import matplotlib.pyplot as plt

In [6]:
name={0:'面包',1:'奶制品',2:'甜品',3:'鸡蛋',4:'油炸食品',5:'肉',6:'面条',7:'米饭',8:'海鲜',9:'汤',10:'水果蔬菜'}

In [7]:
trainx,trainy=readfile(r'F:\mldata\hw3\food-11\training',True)

In [8]:
trainy=np_utils.to_categorical(trainy,11)

In [9]:
valx,valy=readfile(r'F:\mldata\hw3\food-11\validation',True)

In [10]:
valy=np_utils.to_categorical(valy,11)

In [11]:
train_val_x = np.concatenate((trainx, valx), axis=0)
train_val_y = np.concatenate((trainy, valy), axis=0)

In [13]:
e=np.arange(len(train_val_y))
np.random.shuffle(e)
train_val_x=train_val_x[e]
train_val_y=train_val_y[e]

In [14]:
s=np.arange(len(trainy))
np.random.shuffle(s)
trainx=trainx[s]
trainy=trainy[s]

In [15]:
inp=Input(shape=(128,128,3))
x=Conv2D(64,(3,3),activation='relu')(inp)
x=MaxPool2D(pool_size=(2,2),strides=2)(x)
x=Conv2D(128,(3,3),activation='relu')(x)
x=MaxPool2D((2,2),strides=2)(x)
x=Conv2D(256,(3,3),activation='relu')(x)
x=MaxPool2D((2,2),strides=2)(x)
x=Conv2D(512,(3,3),activation='relu')(x)
x=MaxPool2D((2,2),strides=2)(x)
x=Conv2D(512,(3,3),activation='relu')(x)
x=MaxPool2D((2,2),strides=2)(x)
x=Flatten()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(512,activation='relu')(x)
x=Dense(11,activation='softmax')(x)
model=Model(inp,x)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])




In [16]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 64)      1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 256)       295168    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 256)       0   

In [17]:
modelfilepath='bestCNN.hdf5'
earlystop=EarlyStopping(monitor='val_loss',patience=20,verbose=1,mode='auto')
checkpoint=ModelCheckpoint(modelfilepath,monitor='val_loss',verbose=1,save_best_only=True,mode='auto')
reduce_lr=ReduceLROnPlateau(monitor='val_loss',patience=10,factor=0.1,min_delta=0.00001,cooldown=0,min_lr=0)
history=model.fit(train_val_x,train_val_y,validation_split=0.1,epochs=20,batch_size=32,callbacks=[checkpoint,reduce_lr,earlystop])


Train on 11966 samples, validate on 1330 samples
Epoch 1/20
11966/11966 [==============================] - 954s 80ms/step - loss: 2.7233 - accuracy: 0.1696 - val_loss: 2.1426 - val_accuracy: 0.2120

Epoch 00001: val_loss improved from inf to 2.14262, saving model to bestCNN.hdf5
Epoch 2/20
11966/11966 [==============================] - 936s 78ms/step - loss: 2.0596 - accuracy: 0.2742 - val_loss: 1.9154 - val_accuracy: 0.3436

Epoch 00002: val_loss improved from 2.14262 to 1.91542, saving model to bestCNN.hdf5
Epoch 3/20
11966/11966 [==============================] - 945s 79ms/step - loss: 1.8660 - accuracy: 0.3502 - val_loss: 1.7349 - val_accuracy: 0.3880

Epoch 00003: val_loss improved from 1.91542 to 1.73489, saving model to bestCNN.hdf5
Epoch 4/20
11966/11966 [==============================] - 947s 79ms/step - loss: 1.7069 - accuracy: 0.4107 - val_loss: 1.7454 - val_accuracy: 0.3992

Epoch 00004: val_loss did not improve from 1.73489
Epoch 5/20
11966/11966 [=========================

In [38]:
from keras.utils import plot_model
import pydotplus
plot_model(model, "my_first_model_with_shape_info.png", show_shapes=True)

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [26]:
! pip list

Package                            Version            
---------------------------------- -------------------
absl-py                            0.9.0              
alabaster                          0.7.11             
anaconda-client                    1.7.2              
anaconda-navigator                 1.9.2              
anaconda-project                   0.8.2              
appdirs                            1.4.3              
asn1crypto                         0.24.0             
astor                              0.8.0              
astroid                            2.0.4              
astropy                            3.0.4              
atomicwrites                       1.2.1              
attrs                              18.2.0             
Automat                            0.7.0              
Babel                              2.6.0              
backcall                           0.1.0              
backports.shutil-get-terminal-size 1.0.0              
beautifuls

You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
